In [4]:
# pipでインストールされているかテスト
import pandas as pd
pd.__version__

'2.2.0'

In [5]:
# pineconeのセットアップ
import pinecone

In [2]:
# 初期化
from pinecone import Pinecone, ServerlessSpec, PodSpec
pc = Pinecone(api_key='1a0e3dd8-12dd-42cc-92ef-16e20a38cb4b')

/opt/homebrew/Caskroom/miniforge/base/envs/vectordb/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# インデックスの作成
pc.create_index(
    name="command-test", 
    dimension=3, 
    metric="euclidean",
    spec=PodSpec(
        environment='gcp-starter', 
        pod_type='starter'
    )
)

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '1783031fa0128c2a798a27b8cb4b2d0c', 'Date': 'Tue, 23 Jan 2024 15:04:52 GMT', 'Server': 'Google Frontend', 'Content-Length': '128', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"RESOURCE_EXHAUSTED","message":"Capacity Reached. Projects only support a single Starter index."},"status":429}


In [4]:
# インデックスのリストを取得
pc.list_indexes()

{'indexes': [{'dimension': 3,
              'host': 'command-test-jhwhoin.svc.gcp-starter.pinecone.io',
              'metric': 'euclidean',
              'name': 'command-test',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [5]:
# インデックスに接続
index = pc.Index(name="command-test")

In [6]:
# データの挿入（float）
index.upsert([
    ("X", [1.0, 0.0, 0.0]),
    ("Y", [0.0, 1.0, 0.0]),
    ("Z", [0.0, 0.0, 1.0])
])

{'upserted_count': 3}

In [7]:
# ベクトルの取得
# 原点をリクエスト
index.query(
    vector=[0.0, 0.0, 0.0],
    top_k=3,
    include_values=True
)

{'matches': [{'id': 'Y', 'score': 1.0, 'values': [0.0, 1.0, 0.0]},
             {'id': 'Z', 'score': 1.0, 'values': [0.0, 0.0, 1.0]},
             {'id': 'X', 'score': 1.0, 'values': [1.0, 0.0, 0.0]}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [8]:
# ベクトルの取得
# (1, 0, 0)をリクエスト
index.query(
    vector=[1.0, 0.0, 0.0],
    top_k=3,
    include_values=True
)

{'matches': [{'id': 'X', 'score': 0.0, 'values': [1.0, 0.0, 0.0]},
             {'id': 'Z', 'score': 2.0, 'values': [0.0, 0.0, 1.0]},
             {'id': 'Y', 'score': 2.0, 'values': [0.0, 1.0, 0.0]}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [9]:
# インデックスの削除
pc.delete_index("command-test")